In [ ]:
'''Interactive run file for the G2-structure learning'''
#...ensure this notebook is using the correct kernel for your virtual environment
# Import libraries
import os
import yaml
import numpy as np
import tensorflow as tf

# Import functions
from models.model import GlobalModel, PatchSubModel
from sampling.sampling import LinkSample
from geometry.compression import form_to_vec, vec_to_form, vec_to_metric
from geometry.patches import PatchChange_Coords, PatchChange_G2form, PatchChange_G2metric
from geometry.geometry import exterior_derivative

# Print the hyperparameters
with open(os.getcwd()+'/hyperparameters/hps.yaml', "r") as file:
    hp = yaml.safe_load(file)
print(f'Run hyperparameters:\t...edit in hyperparameters/hps.yaml\n{hp}')

In [ ]:
# Run the script
model_name = 'test'
!python3 -m run {model_name}

In [ ]:
# Import the trained model
#loaded_model_name = model_name #...or set to desired name if not model trained above
#loaded_model_path = os.getcwd() + f"/runs/link_model_{loaded_model_name}.keras" #...reimport the model
loaded_model_path = os.getcwd() + f"/models/link_models/link_model_3form.keras" #...import a pre-trained model instead

# Load the model
loaded_model = tf.keras.models.load_model(loaded_model_path, custom_objects={'GlobalModel': GlobalModel, 'PatchSubModel': PatchSubModel})


In [ ]:
# Test loaded model on fake link points
# Testing hyperparameters
testsize = 100 #...how many link points to use in the testing
metric = False #...whether to learn the G2 3-form (False), or the G2 metric (True)

# Generate the testing data on the Link
test_linkpts, test_link_outputs = LinkSample(testsize, metric=metric)
test_linkpts_tf = tf.convert_to_tensor(test_linkpts)
test_link_outputs_tf = tf.convert_to_tensor(test_link_outputs)
if hp["n_patches"] > 1:
    # Coordinates
    test_linkpts_tf_otherpatches += [PatchChange_Coords(
        test_linkpts_tf[0], 
        output_patch=o_patch,
        ) for o_patch in range(1,5)
        ]
    test_linkpts_tf = tf.stack([test_linkpts_tf] + test_linkpts_tf_otherpatches, axis=0)
    # 3-form
    if not metric:
        test_link_outputs_tf_otherpatches += [PatchChange_G2form(
            test_linkpts_tf[0], 
            test_link_outputs_tf[0],
            output_patch=o_patch,
            ) for o_patch in range(1,5)
            ]
        test_link_outputs_tf = tf.stack([test_link_outputs_tf] + test_link_outputs_tf_otherpatches, axis=0)
    # Metric
    else:
        test_link_outputs_tf += [PatchChange_G2metric(
            test_linkpts_tf[0], 
            test_link_outputs_tf[0],
            output_patch=o_patch,
            ) for o_patch in range(1,5)
            ]
        test_link_outputs_tf = tf.stack([test_link_outputs_tf] + test_link_outputs_tf_otherpatches, axis=0)
else:
    test_linkpts_tf = tf.expand_dims(test_linkpts_tf, axis=0)
    test_link_outputs_tf = tf.expand_dims(test_link_outputs_tf, axis=0)
    
# Compute the NN test outputs      
if not hp["metric"]:
    # Compute the predicted G2 3-forms
    predicted_g2form_vecs = np.array([loaded_model.patch_submodels[patch_idx].predict(test_linkpts_tf[patch_idx]) for patch_idx in range(loaded_model.n_patches)])
    predicted_g2forms = np.array([vec_to_form(predicted_g2form_vecs[patch_idx], 7, 3).numpy() for patch_idx in range(loaded_model.n_patches)])
    print(f'G2 3-forms computed... (shape: {predicted_g2forms.shape})')
else:
    # Compute the predicted G2 metrics
    predicted_g2metric_vecs = np.array([loaded_model.patch_submodels[patch_idx].predict(test_linkpts_tf[patch_idx]) for patch_idx in range(loaded_model.n_patches)])
    predicted_g2metrics = np.array([vec_to_metric(predicted_g2form_vecs[patch_idx], 7, 3).numpy() for patch_idx in range(loaded_model.n_patches)])
    print(f'G2 metrics computed... (shape: {predicted_g2metrics.shape})')

# Compute the MSE loss between computed and NN-predicted values
if not hp["metric"]:
    full_mse_loss = np.mean((test_link_outputs_tf.numpy() - predicted_g2forms) ** 2)
    test_outputs_vecs = np.array([form_to_vec(tsm) for tsm in test_link_outputs_tf])
    vec_mse_loss = np.mean((test_outputs_vecs - predicted_g2form_vecs) ** 2)
else:
    full_mse_loss = np.mean((test_link_outputs_tf.numpy() - predicted_g2metrics) ** 2)
    test_outputs_vecs = np.array([form_to_vec(tsm) for tsm in test_link_outputs_tf])
    vec_mse_loss = np.mean((test_outputs_vecs - predicted_g2metric_vecs) ** 2)
print(f'Test losses (full-form, dof-vector): {(full_mse_loss, vec_mse_loss)}')


In [ ]:
# Compute exterior derivative of 3-forms over the test data
assert hp["metric"] == False, "Exterior derivative only configured for the G2 3-form."

# Compute d\phi
dg2_3form = np.array([exterior_derivative(loaded_model.patch_submodels[patch_idx], test_linkpts_tf[patch_idx]) for patch_idx in range(loaded_model.n_patches)])
print(f'G2 3-form exterior derivatives computed... (shape: {dg2_3form.shape})')

# Run general check
###compute omega ^ omega, compare to dg2_3form
###print(f'Non-zero elements: {np.sum(np.where(np.absolute(np.mean(dg2_3form[0],axis=0)) < 1e-5, 1, 0))} / {np.prod(dg2_3form.shape[2:])}')


In [ ]:
########################################################################################

In [ ]:
###testinggg
from geometry.geometry import wedge_product, wedge_form2_with_form1
fake_batchsize = 10
'''#tf
fake_2_forms = tf.random.normal((fake_batchsize, 6, 6))
fake_2_forms = fake_2_forms - tf.transpose(fake_2_forms, perm=[0, 2, 1])
fake_1_forms = tf.random.normal((fake_batchsize, 7))
'''
#np
fake_2_forms = np.random.randn(fake_batchsize, 6, 6)
fake_2_forms_66 = fake_2_forms - np.transpose(fake_2_forms, axes=[0, 2, 1])
fake_2_forms_77 = np.pad(fake_2_forms_66, ((0,0), (0,1), (0,1)), mode='constant')
fake_1_forms = np.random.normal(size=(fake_batchsize, 7))
print(f'Data shapes: {fake_2_forms.shape}, {fake_1_forms.shape}')

# old functionality:
output_old = wedge_form2_with_form1(fake_2_forms_66, fake_1_forms)
output_new = np.array([wedge_product(fake_2_forms_77[idx], fake_1_forms[idx]) for idx in range(fake_1_forms.shape[0])])

# scale output_old to match output_new
output_old *= 3 ###why is there this factor of 3 difference?

print(f'Output shapes: {output_old.shape}, {output_new.shape}')
print(f'Matching?? --> {np.allclose(output_old, output_new)}')